# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

### If on runpod / vast.ai / etc, spin up an A6000 or A100 pod using a Stable Diffusion template with Jupyter pre-installed.

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [ ]:
#BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets

In [ ]:
## Login to stable diffusion
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
## Download the 1.4 sd model
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

In [ ]:
## Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt

# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 400 images
!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 400 \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt "person"

# If you don't want to train against "person", change it to whatever you want above, and on some of the cells below:

In [ ]:
# zip up the files for downloading and reuse.
!apt-get install -y zip
!zip -r all_images.zip outputs/

# Download this file locally so you can reuse during another training on this dataset

## Download pre-generated regularization images

We've created the following image sets

* man_euler - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* man_unsplash - pictures from various photographers
* person_ddim
* woman_ddim - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended

In [1]:
# Grab the existing regularization images
# Choose the dataset that best represents what you are trying to do and matches what you used for your token
# man_euler, man_unsplash, person_ddim, woman_ddim
dataset="person_ddim"
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p outputs/txt2img-samples/samples/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* outputs/txt2img-samples/samples/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_samples
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body 
*   5-12 close-up on face

In [ ]:
#@markdown Add here the URLs to the images of the concept you are adding
urls = [
 "https://i.imgur.com/test1.png",
 "https://i.imgur.com/test2.png",
 "https://i.imgur.com/test3.png",
 "https://i.imgur.com/test4.png",
 "https://i.imgur.com/test5.png",
 ## You can add additional images here
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_samples"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

## Edit the personalized.py file
Execute this cell `%load ldm/data/personalized.py`

Change `joepenna` to whatever you want it to be (but keep the {})

```
training_templates_smallest = [
    'joepenna {}',
]
```

Then paste this at the very top of the cell

```
%%writefile ldm/data/personalized.py
```

Then run the cell again.  This will save your changes.


In [ ]:
%load ldm/data/personalized.py

In [ ]:
# START THE TRAINING
project_name = "project_name"
batch_size = 1000
class_word = "woman"  # << match this word to the class word from regularization images above
reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples/" + dataset

!rm -rf training_samples/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root {reg_data_root} \
 -n {project_name} \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_samples" \
 --batch_size {batch_size} \
 --class_word class_word

## Pruning (12GB to 2GB)
We are working on having this happen automatically (TODO: PR's welcome)

In [ ]:
directory_paths = !ls -d logs/*

In [ ]:
# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
training_samples = !ls training_samples
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(batch_size) + "_batch_size_" + class_word + "_class_word.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{file_name}

In [ ]:
# Download your trained model file from `trained_models` and use in your favorite Stable Diffusion repo!

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/" + {file_name} \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"